### BETTER WAY 35 <br/> 제너레이터 안에서 throw로 상태를 변화시키지 말자

- 제너레이터 고급기능으로 throw 메서드가 있습니다.

In [9]:
class MyError(Exception):
    pass

In [10]:
def my_generator():
    yield 1
    yield 2
    yield 3

In [11]:
it = my_generator()
print(next(it))  # 1을 내놓음
print(next(it))  # 2를 내놓음
print(it.throw(MyError('test error')))

1
2


MyError: test error

- yield까지 코드를 실행할 때 1과 2를 내놓은 후로 throw메소드를 통해 <br/> test error를 발생시키고 1, 2까지 출력되고 error가 뜬다.

In [12]:
def my_generator():
    yield 1
    try:
        yield 2
    except MyError:
        print('MyError 발생!')
    else:
        yield 3
    yield 4

it = my_generator()
print(next(it))  # 1을 내놓음
print(next(it))  # 2를 내놓음
print(it.throw(MyError('test error')))

1
2
MyError 발생!
4


Q. 코드가 중단되지 않고 계속 진행되게 하는 방법
<br/> A. try except 구문을 사용
<br/> 
- 3이 나와야 할 자리에 에러가 뜨는 것을 보아 코루틴에서 
<br/> Error가 제너레이터로 잘 주입되었다는 것을 확인했고 에러가 없으면 3이 리턴되겠죠?
- 제너레이터는 try/except 복합문을 사용해 마지막으로 실행된 yield 문을 둘러쌈으로써 이 예외를 잡아낼 수 있습니다. <br/> → 제너레이터와 제너레이터를 호출하는 쪽 사이에 양방향 통신 수단을 제공한다.

---

Ex. 간헐적으로 재설정할 수 있는 타이머가 필요하다고 하면

In [1]:
class Reset(Exception):
    pass

def timer(period):
    current = period
    while current:
        current -= 1
        try:
            yield current
        except Reset:
            current = period

- yield 식에서 Reset 예외가 발생할 때마다 카운터가 period로 재설정된다.
<br/> 매초 폴링되는 외부 입력과 재설정 되는 이벤트를 연결가능하다.

In [4]:
RESETS = [
    False, False, False, True, False, True, False,
    False, False, False, False, False, False, False]

def check_for_reset():
    # 외부 이벤트를 폴링한다
    return RESETS.pop(0)

def announce(remaining):
    print(f'{remaining} 틱 남음')

def run():
    it = timer(4)
    while True:
        try:
            if check_for_reset():
                current = it.throw(Reset())
            else:
                current = next(it)
        except StopIteration:
            break
        else:
            announce(current)

- timer 제너레이터를 구동시키는 run함수를 정의할 수 있다.
<br/> → run 함수는 : coroutine 대신 함수를 통해 예외를 주입하는 코드 주입

In [12]:
run()

3 틱 남음
2 틱 남음
1 틱 남음
3 틱 남음
2 틱 남음
3 틱 남음
2 틱 남음
1 틱 남음
0 틱 남음


- RESET에서 True일때마다 period로 재설정되는 것을 확인 했고
<br/> Q. 내포 단계마다 StopIteration 예외, throw, next나 announce 호출 때문에 내포 관계가 복잡하다아
<br/><br/> A. Run 함수를 간단하게 표현하기 위해 timer함수를 컨테이너 객체로 표현하여 제너레이터를 재정의해야 한다. <br/> → 저자가 말하기를 함수를 복잡하게 코딩하지말고 class 지정해서 원하는 속성을 구현해라고 말하고 있는것 같습니다.

In [5]:
class Timer:
    def __init__(self, period):
        self.current = period
        self.period = period

    def reset(self):
        self.current = self.period

    def __iter__(self):
        while self.current:
            self.current -= 1
            yield self.current

In [6]:
def run():
    timer = Timer(4)
    for current in timer:
        if check_for_reset():
            timer.reset()
        announce(current)

run()

3 틱 남음
2 틱 남음
1 틱 남음
0 틱 남음
3 틱 남음
2 틱 남음
3 틱 남음
2 틱 남음
1 틱 남음
0 틱 남음


- Timer 함수를 컨테이너 객체로 표현하면 run 함수를 for문을 통해 표현하니까 편하다아.
- 최종적으로는 throw 함수를 이용하지말고 이터러블 클래스를 지정해주는게 좋다.

### 기억하자
- throw 메서드를 사용하면 제너레이터가 마지막으로 실행한 yield 식의 위치에서 예외를 다시 발생시킬 수 있다.
- throw를 사용하면 가독성이 나빠진다. 예외를 잡아내고 다시 발생시키는 데 준비 코드가 필요하며 내포 단계가 깊어지기 때문이다.
- 더 나은 방법은 iter 메서드를 구현하는 클래스를 사용하기